In [174]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import seaborn as se
from sklearn.metrics import r2_score
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer, KNNImputer
import sklearn.metrics as skm
from sklearn.feature_selection import VarianceThreshold, RFECV, mutual_info_regression, SelectPercentile, SelectKBest, f_regression, RFE, SelectFromModel, SelectorMixin
from sklearn.svm import SVR
from sklearn.decomposition import PCA, KernelPCA
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor,ExtraTreesRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.pipeline import Pipeline
from lightgbm import LGBMRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import normaltest

In [217]:
def corr_fun(X, y):
    df = pd.DataFrame(X)
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    bla = np.array([np.max(upper[column]) for column in upper.columns])
    return bla
X = pd.read_csv("X_train.csv").drop("id", axis=1).to_numpy()
y = pd.read_csv("y_train.csv").drop("id", axis=1).to_numpy()
pipe_pre = Pipeline([
    ('s1', SimpleImputer(strategy='median')),
    ('s10', VarianceThreshold()),
    ('s6', SelectKBest(score_func=corr_fun)),
    ('s11', MinMaxScaler()),
    ('s5', LGBMRegressor(n_jobs=1))
])
grid = {
    's6__k': [230],
    's5__num_leaves': [24],
    's5__subsample_for_bin': [82],
    's5__min_child_samples': [1],
    's5__max_depth': [8],
    's5__colsample_bytree': [0.5],
    's5__n_estimators': [750],
    's5__learning_rate': [0.05],
}
estimator = GridSearchCV(pipe_pre, grid, cv=5, n_jobs=16, scoring="r2").fit(X, y)

In [218]:
X_test = pd.read_csv("X_test.csv").drop("id", axis=1)
pd.DataFrame(estimator.best_estimator_.predict(X_test)).to_csv("rofl.csv", index_label='id', header=['y'])